# Feature Engineering

This notebook shows what I did with the raw data from the experiment. The overall summary is that I went over the data with a sliding window. For each window I calculate a set of features, which are used for the later modelling. I also normalise these features. I try to give some insights of this very high dimensional data in the end with some visualizations.

- TODO: Another possible feature engineering step is using a kahlman filter and or complimentary filter for the data to smooth the curve and split graphitation from it.

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import pickle
import numpy as np
import pandas as pd
import tqdm
from gestureanalysis.constants import Constants
from gestureanalysis.featureengineering.utils import misc_helper as mh
from gestureanalysis.featureengineering.initial_input import InitialInput
from gestureanalysis.featureengineering import preprocessing as pp
from gestureanalysis.featureengineering import window as win

In [19]:
base_path = "/home/jsimon/Documents/thesis/gesture-analysis/data/"
time_groups_path_corrected_pickl = base_path+"transformed/time_added/all/time-and-groups-corrected-all.pkl"
stats_added_base_path = base_path+"transformed/stats_added/all/"
stats_added_path_pickl = stats_added_base_path+"raw_stats-added-all.pkl"
gyro_calibration_path = base_path+'../scripts/gestureanalysis/gyro_offset.txt'

In [4]:
# check working directory and adopt if needed
import os
os.getcwd()

'/home/jsimon/Documents/thesis/gesture-analysis/scripts'

In [5]:
# in case you need to reload, and know it exists:
with open( time_groups_path_corrected_pickl, "rb" ) as users_pickle_file:
    users = pickle.load(users_pickle_file)

In [7]:
def preprocess(data, const):
    data = data.drop(u'63_Magnetometer_X_ignore_double', 1)
    data = data.drop(u'64_Magnetometer_Y_ignore_double', 1)
    data = data.drop(u'65_Magnetometer_Z_ignore_double', 1)
    pp.convert_values(data, const, gyro_calibration_path)
    pp.convolution_filter(data, const)
    pp.compute_orientation_indipendent_accel(data, const)
    return data

In [44]:
# test the labeling:

# case with labels:
d = users['AB73']['glove_merged']
ld = d[['label_automatic', 'label_manual', 'label_dynamic', 'label_static']]
ld = ld.astype('category')
np.where(ld['label_manual'].notnull())

(array([  3349,   3350,   3351, ..., 182438, 182439, 182440]),)

In [45]:
lablframe = ld.iloc[3300:3500]

In [46]:
lablframe['label_manual'][0:5]

2015-08-20 19:13:17.600    NaN
2015-08-20 19:13:17.612    NaN
2015-08-20 19:13:17.624    NaN
2015-08-20 19:13:17.636    NaN
2015-08-20 19:13:17.648    NaN
Name: label_manual, dtype: category
Categories (31, object): [(1) One, (2) Two, (3) Three, (4) Four, ..., Turn, Walking, Waving, Zoom]

In [47]:
lablframe['label_manual'][lablframe['label_manual'].notnull()][0:5]

2015-08-20 19:13:18.188    (2) Two
2015-08-20 19:13:18.200    (2) Two
2015-08-20 19:13:18.212    (2) Two
2015-08-20 19:13:18.224    (2) Two
2015-08-20 19:13:18.236    (2) Two
Name: label_manual, dtype: category
Categories (31, object): [(1) One, (2) Two, (3) Three, (4) Four, ..., Turn, Walking, Waving, Zoom]

In [48]:
counts = lablframe.groupby('label_manual').size()

In [49]:
counts

label_manual
(1) One            0
(2) Two          121
(3) Three          0
(4) Four           0
(5) Five           0
Calling            0
Come here          0
Continue           0
Cutthroat          0
Down               0
Go away            0
Grasp 1            0
Grasp 2            0
Knocking           0
Never mind         0
Point              0
Point to self      0
Push away          0
Scissor            0
Shoot              0
Shoulder pat       0
Swipe left         0
Swipe right        0
Swipe up           0
Talking            0
Thumbs down        0
Thumbs up          0
Turn               0
Walking            0
Waving             0
Zoom               0
dtype: int64

In [50]:
labelInfo = []
labelInfo.append(counts)
labelInfo.append(counts)
newLabels = pd.DataFrame(labelInfo)

In [51]:
newLabels

label_manual,(1) One,(2) Two,(3) Three,(4) Four,(5) Five,Calling,Come here,Continue,Cutthroat,Down,...,Swipe left,Swipe right,Swipe up,Talking,Thumbs down,Thumbs up,Turn,Walking,Waving,Zoom
0,0,121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# case without labels:
lablframe = ld.iloc[500:700]
counts = lablframe.groupby('label_manual').size()

In [53]:
counts

label_manual
(1) One          0
(2) Two          0
(3) Three        0
(4) Four         0
(5) Five         0
Calling          0
Come here        0
Continue         0
Cutthroat        0
Down             0
Go away          0
Grasp 1          0
Grasp 2          0
Knocking         0
Never mind       0
Point            0
Point to self    0
Push away        0
Scissor          0
Shoot            0
Shoulder pat     0
Swipe left       0
Swipe right      0
Swipe up         0
Talking          0
Thumbs down      0
Thumbs up        0
Turn             0
Walking          0
Waving           0
Zoom             0
dtype: int64

In [54]:
labelInfo.append(counts)
newLabels = pd.DataFrame(labelInfo)

In [55]:
newLabels

label_manual,(1) One,(2) Two,(3) Three,(4) Four,(5) Five,Calling,Come here,Continue,Cutthroat,Down,...,Swipe left,Swipe right,Swipe up,Talking,Thumbs down,Thumbs up,Turn,Walking,Waving,Zoom
0,0,121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,121,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
import timeit
import sys
import scipy as sc
import scipy.stats
import scipy.signal
import gc
from gestureanalysis.featureengineering.utils import header_tools as ht

def transform_to_windows(label_data, val_data, const,label_type):
    ht.create_headers(const)
    print(len(const.feature_headers))

    print("flex const index trace info / transform_to_windows:")
    print(len(const.feature_indices['flex']['row_1']))
    print(len(const.feature_indices['flex']['row_2']))

    start_time = timeit.default_timer()

    num_windows = (len(data) - const.window_size) // const.step_size

    print('creating empy frame with len {}'.format(num_windows))

    t1 = timeit.default_timer()
    print(val_data.info())
    #convert_to_float(data)
    #data.select_dtypes(include='int').astype(np.float64, copy=False)
    #data = data.astype('float64')
    data[label_type] = data[label_type].astype("category")
    t2 = timeit.default_timer()
    print("converting to float took {}s".format(t2 - t1))

    matrix = np.zeros(shape=(num_windows,len(const.feature_headers))) #None
    labelInfo = []

    for i in tqdm.tqdm_notebook(range(num_windows)):
        #if i % 20 == 0:
        #    progress = (i / float(num_windows))
        #    msg = '\r[{0}] {1:.2f}%'.format('#' * int(progress * 10), progress * 100)
        #    sys.stdout.write(msg)
        offset = i * const.step_size

        # COMPUTE Features for Single Signals:
        subframe = val_data.iloc[offset:(offset + const.window_size)]
        #subframe = subframe._get_numeric_data()
        mat = win.single_value_features(subframe.values[:, 0], subframe.columns)
        sub_range = range(1, len(subframe.columns))
        for j in sub_range:
            if not subframe.columns[j].startswith('label'):
                vec = win.single_value_features(subframe.values[:, j], subframe.columns)
                mat = np.column_stack((mat, vec))
                # do not forget to add peaks!
        mat = np.array(np.ravel(mat))

        corr_mat = None
        finger_flex_line1_tuples = mh.get_combinations(const.raw_indices['flex']['row_1'])
        for idx1,idx2 in finger_flex_line1_tuples:
            vec = win.correlational_features(subframe.values[:, idx1], subframe.values[:, idx2])
            if corr_mat is None:
                corr_mat = vec
            else:
                corr_mat = np.column_stack((corr_mat, vec))

        finger_flex_line2_tuples = mh.get_combinations(const.raw_indices['flex']['row_2'])
        for idx1, idx2 in finger_flex_line2_tuples:
            vec = win.correlational_features(subframe.values[:, idx1], subframe.values[:, idx2])
            corr_mat = np.column_stack((corr_mat, vec))

        mat = np.append(mat,np.array(np.ravel(corr_mat)))

        # ok, maybe getting alls tuples is dump, maybe it is better to list all meaningful combinations
        # like all finger rows, or all finger IMUs and so on...

        #flex_data = data.ix[offset:offset + const.window_size, Constants.flex_map]
        #row1 = flex_data.ix[:, Constants.hand_row_1]
        #row2 = flex_data.ix[:, Constants.hand_row_2]
        #res = self.computeTupelFeatures(row1, newHeaders, "flex_row1")
        #mat = np.append(mat, res)
        #res = self.computeTupelFeatures(row2, newHeaders, "flex_row2")
        #mat = np.append(mat, res)
        
        #print(subframe[label_type].describe())
        lablframe = label_data.iloc[offset:(offset + const.window_size)]
        counts = lablframe.groupby(label_type).size()
        labelInfo.append(counts)

        matrix[i,:] = mat
        gc.collect()
        # if matrix is None:
        #     matrix = mat
        # else:
        #     matrix = np.vstack((matrix, mat))

    newData = pd.DataFrame(matrix, index=range(num_windows), columns=const.feature_headers)
    newLabels = pd.DataFrame(labelInfo)

    print('\nCleaning ...')

    l = pd.isnull(newData).any(1).nonzero()[0]
    print(l)

    print('Done aggregating')

    time = timeit.default_timer() - start_time
    print('exec took {}s'.format(time))

    return (newData, newLabels)


In [15]:
import pathlib
for user, udata in tqdm.tqdm_notebook(users.items()):
    const = Constants()
    const.remove_stripped_headers()
    if 'glove_merged' not in udata:
        print(f"skipping user {user}, no data")
        continue
    path = f"{stats_added_base_path}{user}-window-data.pkl"
    if pathlib.Path(path).exists():
        print(f"skipping user {user}, data already here")
        continue
    data = udata['glove_merged']
    data = preprocess(data, const)
    label_data = data[['label_automatic', 'label_manual', 'label_dynamic', 'label_static']]
    label_data = label_data.astype('category')
    val_data = data.copy()
    val_data = val_data.drop(['label_automatic', 'label_manual', 'label_dynamic', 'label_static'], axis=1)
    data, labels = transform_to_windows(label_data, val_data, const, 'label_manual')
    with open( path, "wb" ) as users_pickle_file:
        pickle.dump({'windata': data, 'winlabels': labels}, users_pickle_file)

skipping user AB73, data already here
skipping user AE30, no data
skipping user AF82, data already here
skipping user AL29, data already here
skipping user AW18, data already here
skipping user CB23, data already here
[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
['0_Thumb_base', '1_Thumb_pressure', '2_Angle_between_thumb_and_hand', '3_Finger_1_base', '4_Finger_1_tip', '5_Finger_2_base', '6_Finger_2_tip', '7_Finger_3_base', '8_Finger_3_tip', '9_Finger_4_base', '10_Finger_4_tip', '11_Thumb_tip', '12_Finger_1_pressure', '13_Finger_2_pressure', '14_Finger_3_pressure', '15_Finger_4_pressure', '16_Wrist_extension', '17_Wrist_flexion', '18_Finger_1_Accel_X', '19_Finger_1_Accel_Y', '20_Finger_1_Accel_Z', '21_Finger_1_Gyro_X', '22_Finger_1_Gyro_Y', '23_Finger_1_Gyro_Z', '24_Finger_2_Accel_X', '25_Finger_2_Accel_Y', '26_Finger_2_Accel_Z', '27_Finger_2_Gyro_X', '28_Finger_2_Gyro_Y', '29_Finger_2_Gyro_Z', '30_Finger_3_Accel_X', '31_Finger_3_Accel_Y', '32_Fin

7201
flex const index trace info / transform_to_windows:
385
385
creating empy frame with len 3529
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 106082 entries, 2015-08-07 11:07:58 to 2015-08-07 11:29:25.136000
Columns: 119 entries, 0_Thumb_base to 122_Wrist_Gyro_Z_direction_cosine
dtypes: float64(98), int64(21)
memory usage: 97.1 MB
None
converting to float took 0.01311484002508223s


/home/jsimon/Documents/thesis/gesture-analysis/scripts/gestureanalysis/featureengineering/window.py:183: RuntimeWarning: invalid value encountered in double_scalars
  spectral_centroid = np.sum(magnitudes * freqs) / np.sum(magnitudes)
/home/jsimon/Documents/thesis/gesture-analysis/scripts/gestureanalysis/featureengineering/window.py:186: RuntimeWarning: invalid value encountered in true_divide
  psdnorm = psd / psdsum



Cleaning ...
[  56   57   58 ... 3526 3527 3528]
Done aggregating
exec took 2514.8649632139714s


/home/jsimon/anaconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


...

(feature) fatal: sensor index not found 66 (new: 3939)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_45_Finger_2_Accel_X_lin_accel_cwt_min)
(feature) fatal: hand index not found 66 (new: 3940)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_46_Finger_2_Accel_X_lin_accel_cwt_25q)
(feature) fatal: sensor index not found 66 (new: 3940)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_46_Finger_2_Accel_X_lin_accel_cwt_25q)
(feature) fatal: hand index not found 66 (new: 3941)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_47_Finger_2_Accel_X_lin_accel_cwt_median)
(feature) fatal: sensor index not found 66 (new: 3941)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_47_Finger_2_Accel_X_lin_accel_cwt_median)
(feature) fatal: hand index not found 66 (new: 3942)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_48_Finger_2_Accel_X_lin_accel_cwt_75q)
(feature) fatal: sensor inde


Cleaning ...
[]
Done aggregating
exec took 2545.7692568579805s
[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
['0_Thumb_base', '1_Thumb_pressure', '2_Angle_between_thumb_and_hand', '3_Finger_1_base', '4_Finger_1_tip', '5_Finger_2_base', '6_Finger_2_tip', '7_Finger_3_base', '8_Finger_3_tip', '9_Finger_4_base', '10_Finger_4_tip', '11_Thumb_tip', '12_Finger_1_pressure', '13_Finger_2_pressure', '14_Finger_3_pressure', '15_Finger_4_pressure', '16_Wrist_extension', '17_Wrist_flexion', '18_Finger_1_Accel_X', '19_Finger_1_Accel_Y', '20_Finger_1_Accel_Z', '21_Finger_1_Gyro_X', '22_Finger_1_Gyro_Y', '23_Finger_1_Gyro_Z', '24_Finger_2_Accel_X', '25_Finger_2_Accel_Y', '26_Finger_2_Accel_Z', '27_Finger_2_Gyro_X', '28_Finger_2_Gyro_Y', '29_Finger_2_Gyro_Z', '30_Finger_3_Accel_X', '31_Finger_3_Accel_Y', '32_Finger_3_Accel_Z', '33_Finger_3_Gyro_X', '34_Finger_3_Gyro_Y', '35_Finger_3_Gyro_Z', '36_Finger_4_Accel_X', '37_Finger_4_Accel_Y', '38_Finger_4_Accel_Z', '39

(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_48_Finger_2_Accel_X_lin_accel_cwt_75q)
(feature) fatal: sensor index not found 66 (new: 3942)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_48_Finger_2_Accel_X_lin_accel_cwt_75q)
(feature) fatal: hand index not found 66 (new: 3943)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_49_Finger_2_Accel_X_lin_accel_cwt_max)
(feature) fatal: sensor index not found 66 (new: 3943)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_49_Finger_2_Accel_X_lin_accel_cwt_max)
(feature) fatal: hand index not found 66 (new: 3944)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_50_Finger_2_Accel_X_lin_accel_cwt_range)
(feature) fatal: sensor index not found 66 (new: 3944)
(feature) fatal: header: 70_Finger_2_Accel_X_lin_accel (feature: 70_50_Finger_2_Accel_X_lin_accel_cwt_range)
(feature) fatal: hand index not found 66 (new: 3945)
(feature) fatal: header: 70_Fi


Cleaning ...
[ 279  280  281 ... 3430 3431 3432]
Done aggregating
exec took 2453.849842828s
[67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
['0_Thumb_base', '1_Thumb_pressure', '2_Angle_between_thumb_and_hand', '3_Finger_1_base', '4_Finger_1_tip', '5_Finger_2_base', '6_Finger_2_tip', '7_Finger_3_base', '8_Finger_3_tip', '9_Finger_4_base', '10_Finger_4_tip', '11_Thumb_tip', '12_Finger_1_pressure', '13_Finger_2_pressure', '14_Finger_3_pressure', '15_Finger_4_pressure', '16_Wrist_extension', '17_Wrist_flexion', '18_Finger_1_Accel_X', '19_Finger_1_Accel_Y', '20_Finger_1_Accel_Z', '21_Finger_1_Gyro_X', '22_Finger_1_Gyro_Y', '23_Finger_1_Gyro_Z', '24_Finger_2_Accel_X', '25_Finger_2_Accel_Y', '26_Finger_2_Accel_Z', '27_Finger_2_Gyro_X', '28_Finger_2_Gyro_Y', '29_Finger_2_Gyro_Z', '30_Finger_3_Accel_X', '31_Finger_3_Accel_Y', '32_Finger_3_Accel_Z', '33_Finger_3_Gyro_X', '34_Finger_3_Gyro_Y', '35_Finger_3_Gyro_Z', '36_Finger_4_Accel_X', '37_Finger_4_Accel_Y


Cleaning ...
[]
Done aggregating
exec took 4557.256682988023s
